# Echo TTS Server - Colab

Run Echo TTS with Cloudflare tunnel for Pipecat.


In [ ]:
# Step 1: Check GPU
import torch
print(f"GPU: {torch.cuda.get_device_name(0)}" if torch.cuda.is_available() else "⚠️ No GPU!")


In [ ]:
# Step 2: Clone repo & install
import os, subprocess, sys

REPO_URL = "https://github.com/PranavGovindu/echopipeline.git"
REPO_DIR = "/content/echo-tts-api"

if not os.path.exists(REPO_DIR):
    subprocess.run(["git", "clone", "--depth", "1", REPO_URL, REPO_DIR], check=True)
    print("✅ Cloned repo")
else:
    subprocess.run(["git", "-C", REPO_DIR, "pull"], check=True)
    print("✅ Updated repo")

# Install deps
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "-r", f"{REPO_DIR}/requirements.txt"], check=True)
print("✅ Installed dependencies")


In [ ]:
# Step 3: Upload your voice file (elise.wav)
from google.colab import files
import shutil

print("📤 Upload your voice file (e.g., elise.wav):")
uploaded = files.upload()

for filename in uploaded.keys():
    dest = f"{REPO_DIR}/audio_prompts/{filename}"
    shutil.move(filename, dest)
    voice_name = filename.rsplit('.', 1)[0]
    print(f"✅ Voice added: {voice_name}")
    print(f"   Use in requests: \"voice\": \"{voice_name}\"")


In [ ]:
# Step 4: Download cloudflared
import urllib.request, stat

urllib.request.urlretrieve(
    "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64",
    "/content/cloudflared"
)
os.chmod("/content/cloudflared", stat.S_IRWXU)
print("✅ Downloaded cloudflared")


In [ ]:
# Step 5: Start server + Cloudflare tunnel
import subprocess, re, time, threading

os.chdir(REPO_DIR)
env = os.environ.copy()
env["ECHO_DEVICE"] = "cuda"
env["PORT"] = "8000"

# Latency optimizations
env["ECHO_COMPILE"] = "1"              # 100-200ms faster (2-3min startup)
env["ECHO_PERFORMANCE_PRESET"] = "low_mid"  # 50-100ms faster
env["ECHO_CACHE_SPEAKER_ON_GPU"] = "1" # 20-60ms faster per request
env["ECHO_WARMUP_VOICE"] = "elise"     # Pre-warm with your voice
env["ECHO_FISH_DTYPE"] = "bfloat16"    # 20-50ms faster decoder

print("🚀 Starting Echo TTS server...")
srv = subprocess.Popen(f"python {REPO_DIR}/api_server.py", shell=True, 
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, env=env)

print("🌐 Starting Cloudflare tunnel...")
cf = subprocess.Popen("/content/cloudflared tunnel --url http://localhost:8000 --no-autoupdate",
    shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)

public_url = None
url_pattern = re.compile(r"(https://[a-z0-9-]+\.trycloudflare\.com)")

def read_srv():
    for line in srv.stdout:
        if line.strip(): print(f"[SERVER] {line.strip()}")

def read_cf():
    global public_url
    for line in cf.stdout:
        m = url_pattern.search(line)
        if m: public_url = m.group(1); break

threading.Thread(target=read_srv, daemon=True).start()
threading.Thread(target=read_cf, daemon=True).start()

print("\n⏳ Waiting for tunnel...")
while not public_url: time.sleep(0.5)

print("=" * 60)
print(f"✅ PUBLIC URL: {public_url}")
print("=" * 60)
print(f"\n📋 For Pipecat: ECHO_SERVER_URL={public_url.replace('https://', 'wss://')}")
print(f"\n🎤 Your voice: elise")
print("=" * 60)

# Keep running
while srv.poll() is None and cf.poll() is None: time.sleep(1)
